In [2]:
def summary(file_name,day, yeni = 0 ):
  file = pd.ExcelFile(file_name)
  df = pd.read_excel(file,"ARAMA LİSTELERİ V.2", header = 1)
  df = df.iloc[(df["FİRMA ADI"].dropna().index)]
  try:
   df_1 = pd.read_excel(file,"VERİLEN TEKLİFLER1")
  except:
   df_1 = pd.read_excel(file,"VERİLEN TEKLİFLER")

  ####################################### 
  df.columns = [column.strip() for column in df.columns]
  today = day
########################################  Functıon  #######################################
####################################### Company Count
  def company_count(df):
    li = []
    li2 = []

    for i in df.columns:
      if i.startswith("İRTİBAT TARİHİ"):
        company_count = df['FİRMA ADI'][df[i]== today]
        li.append(company_count)
        
    return pd.concat(li).count()

####################################### Date
  def rewrite_date(series):
    return ((series[1:]).dropna()).apply(lambda x:str(x).split(" ")[0])
#######################################
  def country_count(df):
    li = []

    for i in df.columns:
      if i.startswith("İRTİBAT TARİHİ"):
        country_count = df['ÜLKE'][df[i]== today]
        li.append(country_count)
    return pd.concat(li).value_counts()



####################################### contact_method
  def methods_count(method):
      li = [0,0,0,0]
      for i in method.itertuples():
        if i[1] == 'ARAMA':
          li[0]+=1
        elif i[1] == 'WHATSAPP':
          li[1]+=1
        elif i[1] == 'E-POSTA':
          li[2]+=1
        elif i[1] == 'DİĞER':
          li[3]+=1
      return li
#######################################
  def contact_method(df):
    li_date = [i for i in  df.columns if i.startswith("İRTİBAT TARİHİ") ]
    li_methods = [i for i in  df.columns if i.startswith("İRTİBAT ŞEKLİ") ]

    li = []
    for i in range(len(li_methods)):
      method = pd.DataFrame(df[li_methods[i]].loc[(df[li_date[i]]==today)])
      li.append(method)
    methods_count_list = pd.Series(li).apply(lambda x: np.array(methods_count(x))).sum()
    return methods_count_list
  def times_count(df):
    li_date = [i for i in  df.columns if i.startswith("İRTİBAT TARİHİ") ]
    li_times = [i for i in  df.columns if i.startswith('KAÇINCI İRTİBA') ]

    li = []
    for i in range(len(li_times)):
      time = pd.DataFrame(df[li_times[i]].loc[(df[li_date[i]]==today)])
      li.append(time.dropna())
    times_count = pd.Series(li).apply(lambda x: np.array(times_count(x))).sum()
    return times_count
  


#######################################
  def result_cat(s):
      arama_katalog = ((s=="HATIRLATMA VE SORUYA CEVAP VERME")|(s=="KATALOG WP ")|(s=="KATALOG EMİAL")|(s=="KATALOG DİĞER")|(s=="ŞİMDİLİK OLMUYOR")|(s=="İLGİLENMİYOR")|(s=="SONRA ARA")| (s== "FIYAT YÜKSEK")|(s=="İTHALAT YAPAMAZ")).sum()
      cevap_olmayip_mesaj = ((s=="CEVAP YOK  WP")|(s== "CEVAP YOK  EMAIL")|(s=="CEVAP YOK KATALOG  WP")).sum()
      proje_ve_z = ((s=="PROJE BEKLENİYOR")| (s=="ZIYARETE GELEBİLİR")).sum()
      cevapsiz = (s=="CEVAP YOK").sum()
      return [arama_katalog,cevap_olmayip_mesaj,proje_ve_z,cevapsiz]
  def result(df):
    li_date = [i for i in  df.columns if i.startswith("İRTİBAT TARİHİ") ]
    li_result = [i for i in  df.columns if i.startswith("İRTİBAT SONUCU") ]

    li=[]
    for i in range(len(li_result)):
      result = pd.DataFrame(df[li_result[i]].loc[(df[li_date[i]]==today)])
      li.append(result)


    result_count_list = pd.Series(li).apply(lambda x: np.array(result_cat(x))).sum()
    return result_count_list
    



####################################### contact times
  def times_count(d):  
      contact12 =(d<=2).sum()[0]
      contact32 = ((d==3)|(d==4)).sum()[0]
      contact4 = (d>4).sum()[0]
      return [contact12,contact32,contact4]

  def count_times(df):
    li_date = [i for i in  df.columns if i.startswith("İRTİBAT TARİHİ") ]
    li_times = [i for i in  df.columns if i.startswith('KAÇINCI İRTİBA') ]

    li = []
    for i in range(len(li_times)):
      time = pd.DataFrame(df[li_times[i]].loc[(df[li_date[i]]==today)])
      li.append(time.dropna())
    return pd.Series(li).apply(lambda x: np.array(times_count(x))).sum()
 
########################################  :::::::  #######################################

####################################### Person
  if df["EKLEYEN"][0].strip() == "MA" :
    person = "MAHMOUD ATiA"
  elif df["EKLEYEN"][0].strip() == "AMIR":
    person = "Amir"
  else: 
    person =  "ALi ABDULRAZZAQ"
####################################### Qoutes
  df_1['date']= rewrite_date(df_1["VERİLEN TEKLİFLER"])
  qoute = (df_1['date'] ==today).sum()
###################################### Country
  country = pd.DataFrame(country_count(df))
  text = ""
  for count in country.itertuples():
        text+= ("   " + count[0]+ " : " + str(count[1]))
###################################### results
  result = result(df)
###################################### contact methods
  contact = contact_method(df)
###################################### contact times
  times =  count_times(df)
  print (f"""
  Bugün Özeti:
  {person}
  Tarih : {today}
  Toplam Firma : {company_count(df)}
  Yeni Eklenen : {yeni}
  Verilen Teklifler : {qoute}
  Ülkeler:
  {
      text
  }
  ---------------------------------
  Arama : {contact[0]}
  WP : {contact[1]}
  Email : {contact[2]}
  Diğer : {contact[3]}
  ---------------------------------
  Arama Katalog : {int(result[0][0])}
  Cevap olmayıp Katalog : {int(result[1][0])}
  Proje Ziyaret bekleniyor : {int(result[2][0])}
  Cevapsız :{int(result[3][0])}
  ---------------------------------
  1. ve 2. Irtibat : {int(times[0])}
  3. ve 4. Irtibat : {int(times[1])}
  4ten fazla. Irtibat :{int(times[2])}
  ---------------------------------
  """
  )

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
today = datetime.now().strftime("%Y-%m-%d")

In [7]:
file = "/content/(MOUSTAPHA) SIRMA METAL  - İhracat Sistemi - Made by Muhammet AKMAN (2).xlsx"
yeni = 0
summary(file,today,yeni)


  Bugün Özeti:
  Amir
  Tarih : 2022-12-24
  Toplam Firma : 0
  Yeni Eklenen : 0
  Verilen Teklifler : 0
  Ülkeler:
  
  ---------------------------------
  Arama : 0
  WP : 0
  Email : 0
  Diğer : 0
  ---------------------------------
  Arama Katalog : 0
  Cevap olmayıp Katalog : 0
  Proje Ziyaret bekleniyor : 0
  Cevapsız :0
  ---------------------------------
  1. ve 2. Irtibat : 0
  3. ve 4. Irtibat : 0
  4ten fazla. Irtibat :0
  ---------------------------------
  
